In [21]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import StaleElementReferenceException


In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# Define your years, cause types, and measure values
years = ['2011']
#years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', 2018', '2019', '2020', '2021', '2022']
cause_types = ['All Opioids', 'Drug Overdoses']
measure_values = ['Death Rate', 'Percent of Deaths by Cause', 'Age-Adjusted Death Rate']

# Set up the WebDriver
s = Service("/Users/nikitajha/Documents/NikitaFinal/Coding Projects/Selenium/ChromeDriver/chromedriver")
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": "/Users/nikitajha/Documents/NikitaFinal/Coding Projects/Bastos Lab/Source_2_Data_Folder",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
driver = webdriver.Chrome(service=s, options=chrome_options)

# Navigate to the webpage
driver.get("https://oasis.state.ga.us/gis/TrendableMaps/agsDrugODTrend.aspx")
wait = WebDriverWait(driver, 30)

def run_script_measure(measure):
  # Acquire information from measure dropdown
  measure_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "drpMeasure")))
  select = Select(measure_dropdown)
  select.select_by_value(measure)

def run_script_cause(cause):
  cause_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "drpCauseType")))
  select = Select(cause_dropdown)
  select.select_by_value(cause)
  
  # Click the submit button
  submit_button = wait.until(EC.element_to_be_clickable((By.ID, 'imgSubmit')))
  submit_button.click()

  # Wait for the export to Excel button to be clickable
  excel_button = wait.until(EC.element_to_be_clickable((By.ID, 'ImgSaveData')))
  excel_button.click()

  # Go back to home link and wait for the page to load
  driver.get("https://oasis.state.ga.us/gis/TrendableMaps/agsDrugODTrend.aspx")
  wait.until(EC.presence_of_element_located((By.ID, 'lstTime')))
  wait.until(EC.presence_of_element_located((By.ID, "drpMeasure")))
  wait.until(EC.presence_of_element_located((By.ID, "drpCauseType")))

def run_script_year(year): 
  time_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'lstTime')))
  select_year = Select(time_dropdown)
  select_year.deselect_by_value("2022")
  select_year.select_by_value(year)

for year in years:
  for measure in measure_values:
    for cause in cause_types:
      try:
          run_script_measure(measure)
          run_script_year(year)
          run_script_cause(cause)
      except Exception as e:
          print(f"An error occurred: {e}")
          try: 
            run_script_measure(measure)
            run_script_year(year)
            run_script_cause(cause)
          except Exception as e:
            try: 
              print(f"An error 2 occurred: {e}")
              run_script_measure(measure)
              run_script_year(year)
              run_script_cause(cause)
            except Exception as e: 
              continue

driver.quit()


An error occurred: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000100ec6004 chromedriver + 4169732
1   chromedriver                        0x0000000100ebdff8 chromedriver + 4136952
2   chromedriver                        0x0000000100b13500 chromedriver + 292096
3   chromedriver                        0x0000000100b18808 chromedriver + 313352
4   chromedriver                        0x0000000100b1a780 chromedriver + 321408
5   chromedriver                        0x0000000100b1a8ac chromedriver + 321708
6   chromedriver                        0x0000000100b5a6e8 chromedriver + 583400
7   chromedriver                        0x0000000100b4e3b4 chromedriver + 533428
8   chromedriver                        0x0000000100b4ddb4 chromedr

In [43]:
import os
import sqlite3
import pandas as pd

def setup_database():
    # Database connection
    conn = sqlite3.connect('oasis.db')
    c = conn.cursor()
    
    #Create table
    c.execute('''
        CREATE TABLE IF NOT EXISTS OasisData (
            County TEXT,
            Year INTEGER,
            Variable TEXT,
            Value REAL
        )
        ''')
    conn.commit()
    conn.close()
    

def process_and_insert_data(full_path, conn):
    # Read the first row, first cell to extract year, measure, and cause
    first_cell = pd.read_excel(full_path, header=None, nrows=1).iloc[0, 0]
    year, measure, cause = parse_first_cell(first_cell)
    variable = measure + "-" + cause
    # Read the rest of the data skipping the first row
    data = pd.read_excel(full_path, header=1)
    column_titles = data.columns
    # Database cursor
    c = conn.cursor()

    # Iterate over each row and insert data into the database
    for index, row in data.iterrows():
        county = str(row['Geography']).strip()
        value = row[data.columns[1]]  # or the appropriate column name
        
        if len(county.split(" ")) < 2 and county != "nan":
            c.execute('''
            INSERT INTO OasisData (County, Year, Variable, Value) 
            VALUES (?, ?, ?, ?)
            ''', (county, year, variable, value))

    conn.commit()

def export_to_csv():
    conn = sqlite3.connect("oasis.db")
    query = """ 
    SELECT * FROM OasisData 
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    csv_file_path = "oasis_data_temp.csv"
    df.to_csv(csv_file_path, index=False)

def parse_first_cell(cell_value):
    # Implement the logic to parse the year, measure, and cause from the cell value
    # Example: "Death Rate by County of Residence, Drug Overdoses, 2016"
    parts = cell_value.split(',')
    year = parts[-1].strip()
    cause = parts[-2].strip()
    measure = parts[0].strip()
    if 'by County of Residence' in measure:
        measure = measure.replace('by County of Residence', '').strip()

    return year, measure, cause

if __name__ == "__main__":
    setup_database()
    folder_name = "Source_2_Data_Folder"
    conn = sqlite3.connect('oasis.db')
    
    for filename in os.listdir(folder_name):
        if filename.endswith('.xlsx'):
            full_path = os.path.join(folder_name, filename)
            
            #Process and insert data into db
            process_and_insert_data(full_path, conn)
            export_to_csv()
    
    conn.close()
    os.remove('oasis.db')